# Intecrative Maps

### Libraries

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from matplotlib.ticker import ScalarFormatter

import pandas              as pd # The gold standard of Python data analysis, to create and manipulate tables of data
import numpy               as np # The Python module for processing arrays which/Pandas is based on
import matplotlib.pyplot   as plt # The gold standard of Python data visualization, but can be complex to use
import seaborn             as sns; sns.set() # A package to make Matplotlib visualizations more aesthetic
import branca
import geopandas
import datetime

import folium # package for making maps, please make sure to use a version older than 1.0.0.
# from wordcloud import WordCloud # A package that will allow us to make a wordcloud
from scipy.stats import ttest_ind # A module for Python machine learning--we'll stick to T-Tests here
from IPython.display import display
from folium.plugins import TimeSliderChoropleth
# from time_slider_choropleth import TimeSliderChoropleth
%matplotlib inline
plt.rcParams["figure.figsize"] = (8,5)

%matplotlib inline 
sns.set(color_codes=True)

# SISBEN average by 2021 and 2022

## 2021 DATASET

In [5]:
df21 = pd.read_csv('C2021.csv')
x = datetime.datetime(2021, 1, 1)
# df21['year'] = x.year
del df21['Unnamed: 0']

In [6]:
df21.head()

,NOMBRE,clas_count
0,VILLAREAL,28.603235
1,12 DE FEBRERO,13.164630
2,12 DE OCTUBRE,38.528286
3,13 DE JUNIO,50.941577
4,20 DE JULIO,42.955534


## 2021 DATASET CLUSTER

In [3]:
df21c = pd.read_csv('cluster_2021.csv', encoding='iso-8859-1', sep='~')

In [22]:
df21cn = df21c[['nom_barrio','Cluster']]
df21cn.rename(columns = {'nom_barrio':'NOMBRE', 'Cluster':'clas_count'}, inplace = True)
df21cn.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,NOMBRE,clas_count
0,VILLA LINA,0
1,COLORADOS,2
2,ROSALTA,4
3,CAFE MADRID,4
4,EL ROSAL,0


## 2022 DATASET

In [7]:
df22 = pd.read_csv('C2022.csv')
# df22['year'] = datetime.datetime.today().year
del df22['Unnamed: 0']

## 2022 DATASET CLUSTER

In [8]:
df22c = pd.read_csv('cluster_2022.csv', encoding='iso-8859-1', sep='~')

In [23]:
df22cn = df22c[['NOM_BARRIO','Cluster']]
df22cn.rename(columns = {'NOM_BARRIO':'NOMBRE', 'Cluster':'clas_count'}, inplace = True)
df22cn.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,NOMBRE,clas_count
0,EL PABLÓN,0
1,VILLA LINA,0
2,COLORADOS,5
3,EL PABLÓN,2
4,ROSALTA,4


# VIOLENCE DATASET

In [17]:
violencia_g = pd.read_csv('Violencia_G_2015_2022.csv')
violencia_g['BARRIO'] = violencia_g['Barrio'].str.upper()
vio_cn = violencia_g.groupby([ "BARRIO"])["BARRIO"].count().reset_index(name="clas_count")
vio_cn.rename(columns = {'BARRIO':'NOMBRE'}, inplace = True)
vio_cn.head()

,NOMBRE,clas_count
0,BOSQUE NORTE,2
1,BRISAS DEL MUTIS,1
2,CABECERA DEL LLANO,1
3,CAFÉ MADRID,1
4,CAMPO HERMOSO,3


# CRIME DATASET

In [9]:
delitos = pd.read_csv('Delitos_2010_2021.csv')
delitos_cn = delitos.groupby(["BARRIOS_HECHO"])["BARRIOS_HECHO"].count().reset_index(name="clas_count")
delitos_cn. rename(columns = {'BARRIOS_HECHO':'NOMBRE'}, inplace = True)
delitos_cn.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,NOMBRE,clas_count
0,12 DE OCTUBRE,270
1,13 DE JUNIO,34
2,20 DE JULIO,31
3,23 DE JUNIO,88
4,5 DE ENERO,15


#### We append the both datasets

In [10]:
beat_cn = df21.append(df22, ignore_index=True)
beat_cn.head()

,NOMBRE,clas_count
0,VILLAREAL,28.603235
1,12 DE FEBRERO,13.164630
2,12 DE OCTUBRE,38.528286
3,13 DE JUNIO,50.941577
4,20 DE JULIO,42.955534


### Load GEOJSON

In [5]:
# load the shape file
beat_orig = geopandas.read_file("Barrios_Bucaramanga.geojson", driver = "GeoJSON")

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


## Map definition

In [18]:
def folium_map(beat_orig, beat_cn, name_ng, clas_count, title):
    beat_data = beat_orig.join(beat_cn.set_index(name_ng), how = "inner", on = name_ng)
    beat_data.fillna(np.nan, inplace = True)

    # color scheme
    min_cn, max_cn = beat_cn[clas_count].quantile([0.01,0.99]).apply(round, 2)

    colormap = branca.colormap.LinearColormap(
        colors=['white','yellow','orange','red','darkred'],
        #index=beat_cn['count'].quantile([0.2,0.4,0.6,0.8]),b
        vmin=min_cn,
        vmax=max_cn
    )

    colormap.caption=title

    # interactive visualization for beat-specific crime rate in 2017

    m_crime = folium.Map(location=[7.106287, -73.139366],
                            zoom_start=14,
                            tiles="OpenStreetMap")
    style_function = lambda x: {
        'fillColor': colormap(x['properties'][clas_count]),
        'color': 'black',
        'weight':2,
        'fillOpacity':0.5
    }

    stategeo = folium.GeoJson(
        beat_data.to_json(),
        name='Chicago beats',
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=[name_ng, clas_count],
            aliases=['Neighborhood', 'Total class count'], 
            localize=True
        )
    ).add_to(m_crime)

    colormap.add_to(m_crime)
    return m_crime

### Maps

In [13]:
# MAP FOR CRIME
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "Crime count by Neighborhood"
crimemap = folium_map(beat_orig, delitos_cn, name_ng, clas_count, title)

In [14]:
# MAP FOR SISBEN 2021
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "2021 SISBEN Score by Neighborhood"
s21map = folium_map(beat_orig, df21, name_ng, clas_count, title)

In [15]:
# MAP FOR SISBEN 2022
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "2022 SISBEN Score by Neighborhood"
folium_map(beat_orig, df22, name_ng, clas_count, title)
s22map = folium_map(beat_orig, df22, name_ng, clas_count, title)

In [18]:
# MAP FOR CRIME
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "Genre Violece count by Neighborhood"
viomap = folium_map(beat_orig, vio_cn, name_ng, clas_count, title)

In [25]:
# MAP FOR 2021 CLUSTER
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "Neighborhood Cluster by Vulnerability"
cluster2021 = folium_map(beat_orig, df21cn, name_ng, clas_count, title)

In [28]:
# MAP FOR 2021 CLUSTER
clas_count = 'clas_count'
name_ng = 'NOMBRE'
title = "Neighborhood Cluster by Vulnerability"
cluster2022 = folium_map(beat_orig, df22cn, name_ng, clas_count, title)

#### Export maps as HTML

In [19]:
crimemap.save("crimemap.html")

In [20]:
s21map.save("s21map.html")

In [21]:
s22map.save("s22map.html")

In [22]:
viomap.save("viomap.html")

In [27]:
cluster2021.save("cluster2021.html")

In [30]:
cluster2022.save("cluster2022.html")